In [ ]:
import os

import cantera as ct
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
def simulate(model, T, P, mol_frac, time):

    model.TPX = T, P, mol_frac

    reactor = ct.IdealGasReactor(model)
    
    simulation = ct.ReactorNet([reactor])

    simulation.advance(time)

    conditions = pd.Series()
    conditions['time'] = simulation.time
    conditions['temperature0'] = T
    conditions['temperature'] = model.T
    conditions['pressure0'] = P
    conditions['pressure'] = model.P
        
    species = pd.Series()
    for key, value in model.mole_fraction_dict().iteritems():
        species[key] = value
        
    return conditions, species

In [ ]:
# Convert chemkin to cantera
from cantera import ck2cti
directory = '/home/mjliu/Documents/Models/Naphthalene/acetylene/run7/'
filename = 'chem_mod'
chemkin_file = os.path.join(directory, filename + '.inp')
cantera_file = os.path.join(directory, filename + '.cti')
if os.path.exists(cantera_file):
    raise Exception('File already exists')
ck2cti.Parser().convertMech(chemkin_file, outName=cantera_file)

In [ ]:
filename = '/home/mjliu/Documents/Models/Naphthalene/acetylene/run7/chem.cti'

In [ ]:
input_conditions = [
    (900, 2.92E-06),
    (910, 3.88E-06),
    (920, 5.12E-06),
    (930, 6.72E-06),
    (940, 8.75E-06),
    (950, 1.13E-05),
    (960, 1.46E-05),
    (970, 1.87E-05),
    (980, 2.37E-05),
    (990, 3.00E-05),
    (1000, 3.76E-05),
    (1010, 4.69E-05),
    (1020, 5.81E-05),
    (1030, 7.14E-05),
    (1040, 8.72E-05),
    (1050, 1.06E-04),
    (1060, 1.27E-04),
    (1070, 1.51E-04),
    (1080, 1.78E-04),
    (1090, 2.07E-04),
    (1100, 2.39E-04),
    (1110, 2.73E-04),
    (1120, 3.07E-04),
    (1130, 3.40E-04),
    (1140, 3.73E-04),
    (1150, 4.03E-04),
    (1160, 4.29E-04),
    (1170, 4.50E-04),
    (1180, 4.68E-04),
    (1190, 4.80E-04),
    (1200, 4.89E-04),
]

In [ ]:
model = ct.Solution(filename)

In [ ]:
conditions_df = pd.DataFrame()
species_df = pd.DataFrame()

for T, X in input_conditions:
    #T = 1200
    P = 2 * 101325
    mol_frac = {'Ne':0.95, 'C2H2(2)':0.05-X, 'A2_rad1(6)':X}

    conditions, species = simulate(model, T, P, mol_frac, 0.002)
    
    conditions_df = conditions_df.append(conditions, ignore_index=True)
    species_df = species_df.append(species, ignore_index=True)
    
    print 'T = {0:4} Completed!'.format(T)

data = pd.concat([conditions_df, species_df], axis=1)
print 'Simulation Completed!'

In [ ]:
selected = [
    'A2_5(5)',
    'A2(1)',
    'C12H8(768)',
]
selected2 = [
    'C4H4(2237)',
    'C4H6(758)',
    'A1(3)',
    'C2H4(39)',
]

In [ ]:
plt.style.use('seaborn-poster')
#plt.rcParams['axes.labelsize'] = 24
#plt.rcParams['xtick.labelsize'] = 20
#plt.rcParams['ytick.labelsize'] = 20

fig = plt.figure()

colormap = mpl.cm.tab10

new_labels = ['Acenaphthylene-RMG', 'Naphthalene-RMG', '1-Ethynylnaphthalene-RMG']

for i, label in enumerate(selected):
    if i == 0:
        plt.plot(acenaphthylene_expt[0], acenaphthylene_expt[1], 'o', c=colormap(0), label='Acenaphthylene-Lifshitz-expt')
        plt.plot(acenaphthylene_model[0], acenaphthylene_model[1], '--', c=colormap(0), label='Acenaphthylene-Lifshitz-model')
    elif i == 1:
        plt.plot(naphthalene_expt[0], naphthalene_expt[1], 'o', c=colormap(1), label='Naphthalene-Lifshitz-expt')
        plt.plot(naphthalene_model[0], naphthalene_model[1], '--', c=colormap(1), label='Naphthalene-Lifshitz-model')
    elif i == 2:
        plt.plot(naphthylacetylene_model[0], naphthylacetylene_model[1], '--', c=colormap(2), label='1-Ethynylnaphthalene-Lifshitz-model')

    x = data['temperature0']
    y = data[label]/0.0005*100
    #y = [value if value > 1e-8 else np.nan for value in y]
    plt.plot(x, y, c=colormap(i), label=new_labels[i])
    
plt.yscale('log')
plt.xlabel('Temperature (K)')
plt.ylabel('Percent Yield')
plt.legend(bbox_to_anchor=(1.02, 1), loc=2, borderaxespad=0.)

In [ ]:
plt.style.use('seaborn-poster')
#plt.rcParams['axes.labelsize'] = 24
#plt.rcParams['xtick.labelsize'] = 20
#plt.rcParams['ytick.labelsize'] = 20

fig = plt.figure()

colormap = mpl.cm.tab10

new_labels = ['Vinylacetylene-RMG', '1,3-Butadiene-RMG', 'Benzene-RMG', 'Ethene-RMG']

for i, label in enumerate(selected2):
    x = data['temperature0']
    y = data[label]/0.0005*100
    #y = [value if value > 1e-8 else np.nan for value in y]
    plt.plot(x, y, c=colormap(i), label=new_labels[i])

plt.yscale('log')
#plt.ylim([1e-2, 1e2])
plt.xlabel('Temperature (K)')
plt.ylabel('Percent Yield')
plt.legend(bbox_to_anchor=(1.02, 1), loc=2, borderaxespad=0.)

In [ ]:
def read(string):
    lines = string.strip().splitlines()
    t = []
    x = []
    for line in lines:
        tokens = line.split(', ')
        t.append(float(tokens[0]))
        x.append(float(tokens[1]))
    
    return t, x

In [ ]:
acenaphthylene_expt = read("""
915.4268970605044, 0.3415404042277774
921.5134921706965, 0.6715524451826531
928.7821198865349, 0.5607593310313395
934.1343086626282, 0.5360442632233882
943.6822144408309, 0.6229504549550771
934.500888394759, 0.780427904847085
929.9018090003221, 1.0699507430108044
937.5482382768259, 0.9925156358515191
943.6691223075404, 0.8540523704988493
943.6585239139245, 1.1025907776988502
944.7981629451066, 1.300745990643067
950.1403767625025, 1.5813252556028443
958.5467732047672, 1.6295653372450312
967.7256055111648, 1.3813178940615518
970.7742022630688, 1.7046987181440536
961.592876216997, 2.1356344447852025
976.1114286011159, 2.337107180252829
978.0197628869194, 2.4818742411244785
975.3358755623902, 3.0629067035391007
999.38550098191, 5.844145824906985
1009.7052191893268, 5.671141364589379
1014.6665142714644, 6.690346537955973
1013.5131596720732, 7.892721045083737
1021.5523529472887, 5.671141364589379
1023.4475950998016, 8.256625948468923
1037.5590444820814, 16.48032387326401
1045.9947423655199, 8.38161861795969
1047.5103126526117, 11.4910282067327
1053.9884248916783, 18.03505433766841
1058.9621886721875, 15.753965345666082
1059.3350027535043, 19.736455876957965
1067.7625959830011, 12.20281515169512
1064.6784634407375, 23.28344873792214
1070.414688126682, 21.276275019923478
1065.8137384274894, 30.514232219840185
1073.4639083135048, 25.865704200651447
1073.4495693103772, 36.5431409271721
1078.4239565258051, 31.445102733871515
1083.0005922631726, 39.39420133510683
1086.0529296245884, 44.425731144189385
1087.2168826175955, 29.169339185186825
1092.5753057428747, 23.99373617818744
1101.7235897382614, 42.46769871051222
1118.911690443782, 53.20323127577195
1118.1398780145678, 63.71496304689414
1125.3997776415458, 65.65865869604717
1129.6110805166197, 54.82625488288746
1133.4246319136334, 66.65263021381031
1138.778067559564, 61.828806690373085
1146.0342265770307, 69.72574261950383
1170.8712502987294, 76.3035706426307
1197.2413004852401, 75.1656774240269
1204.884612587151, 75.1656774240269
""")

acenaphthylene_model = read("""
899.0473914443949, 0.0938139747458477
924.613210585925, 0.24174668198151125
949.4165688220197, 0.5954943576897983
974.9873754428986, 1.3607186973116348
999.0326368179882, 2.8842481744935187
1025.372762128407, 5.844145824906985
1049.4223875479265, 11.150858882940748
1074.6222503922445, 19.442131782914633
1099.4436882409784, 30.976170954430213
1124.6510323043194, 45.098268663080006
1150.2436591473488, 60.90677146640111
1174.6947766544404, 72.94054515547023
1199.9145894161531, 78.63129573545962
""")

naphthylacetylene_model = read("""
1049.7889672800575, 0.0016234557539947677
1074.9782317307593, 0.0036543140927172055
1099.7896946207957, 0.007404475327995067
1124.6061449901808, 0.013303944489966755
1150.1906671792688, 0.021843128718956658
1175.0133518978398, 0.033771326842741985
1199.8391537910038, 0.04843451725861638
""")

naphthalene_expt = read("""
943.2857142857142, 0.07765334766497867
967.2857142857142, 0.2581661590809044
971.5714285714286, 0.27501682740068584
976.7142857142858, 0.3812726894956135
978.4285714285714, 0.4757135674410709
1010.1428571428571, 0.9240065657365751
1014, 1.1528821020645799
1022.5714285714286, 1.0937085364819383
1023.8571428571429, 1.4690888241257416
1037.5714285714287, 2.3604657956801818
1046.142857142857, 2.102105276036458
1060.2857142857142, 2.0800692950143547
1065.857142857143, 2.3604657956801818
1066.7142857142858, 3.13738275971436
1047.857142857143, 3.7926901907322508
1054.7142857142858, 3.997888371280274
1059.4285714285713, 5.3700350051598065
1068.4285714285716, 6.158482110660267
1071, 6.356289385961481
1072.2857142857142, 5.5425177652873545
1074, 4.489251258218608
1084.2857142857142, 4.732135897849963
1080, 6.988654854126644
1087.2857142857142, 7.062691746120908
1093.2857142857142, 7.4448087720823
1101.857142857143, 4.126298487843327
1118.5714285714287, 6.356289385961481
1119.4285714285713, 7.062691746120908
1126.7142857142858, 6.6299506764795675
1134.4285714285713, 7.603382956801465
1128.857142857143, 10.87960861640183
1137.857142857143, 10.87960861640183
1145.5714285714284, 6.70018750350959
1170.857142857143, 9.288880734072603
1197.857142857143, 10.765559690436053
1205.142857142857, 10.430536235688859
""")

naphthalene_model = read("""
900, 0.011407966521057087
925.2857142857142, 0.030397476305387174
950.1428571428571, 0.07603382956801465
975.4285714285714, 0.17853207274376864
1000.2857142857142, 0.39351897202119607
1025.142857142857, 0.822871096239495
1050.2142857142858, 1.615243441329967
1075.2857142857142, 2.9763514416313184
1100.142857142857, 5.041005397797906
1124.7857142857142, 7.806358870258297
1150.0714285714284, 10.652706318278561
1174.2857142857142, 13.014170014555933
1200, 15.242856516212672
""")